<h1 style="float: left; margin-left: 1em; margin-top: 0.6em;">Compare the AutoAI model with the reference one and select the winning</h1>

This notebook contains steps and code to demonstrate how to compare a model trained using AutoAI experiment with a reference one, trained using another notebook code.
Both notebooks are triggered within execution of a Orchestration Pipeline, that passes model paths and evaluation results as environment variables.

Some familiarity with Python is helpful. This notebook uses **Python 3.11** with the <a href="https://pypi.python.org/pypi/ibm-watson-pipelines" target="_blank" rel="noopener no referrer"><code>ibm-watson-pipelines</code></a> package.

### Contents

- [Setup](#setup)
- [Model selection](#select)
- [Storing notebook results](#results)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

- If you don't see a project token inserted in the first cell of this notebook, then click the **More** menu, and choose **Insert project token**.

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Connection to watsonx.ai

Authenticate the watsonx.ai service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use <a href="https://cloud.ibm.com/docs/cli/index.html" target="_blank" rel="noopener no referrer">IBM Cloud CLI</a> to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.

Location of your watsonx.ai Runtime instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the <a href="https://cloud.ibm.com/iam#/users" target="_blank" rel="noopener no referrer"><strong>Users</strong> section of the Cloud console</a>. From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the <a href="https://cloud.ibm.com/apidocs/machine-learning" target="_blank" rel="noopener no referrer"><strong>Endpoint URLs</strong> section of the watsonx.ai Runtime docs</a>. You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance details.

You can also get service specific apikey by going to the <a href="https://cloud.ibm.com/iam/serviceids" target="_blank" rel="noopener no referrer"><strong>Service IDs</strong> section of the Cloud Console</a>. From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

If you cannot connect to Watson pipelines client check if `cpdaas_url` matches host url you can see for this notebook preceded by `api.`

In [1]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [13]:
cpdaas_url = 'https://api.dataplatform.cloud.ibm.com'

## Package installation

In [2]:
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install -U ibm-watson-pipelines | tail -n 1

ERROR: ibm-watson-machine-learning 1.0.53 has requirement ibm-cos-sdk==2.7.*, but you'll have ibm-cos-sdk 2.9.0 which is incompatible.
ERROR: responses 0.12.1 has requirement urllib3>=1.25.10, but you'll have urllib3 1.25.9 which is incompatible.


In [19]:
import os
import numpy as np

from ibm_watsonx_ai import Credentials, APIClient
from ibm_watson_pipelines import WatsonPipelines
from ibm_watson_pipelines.cpd_paths import CpdScope, CpdPath

## Notebook parameters

This notebook is run from the Orchestration Pipeline that passes following environment variables:

- `autoai_model` - the AutoAI model (as CPD Path, e.g. 'cpd:///spaces/b5ad9f7e-eec3-49ad-a8f6-86354ff29134/assets/b6046bcd-7c20-4fc8-bb70-9ed0944d185b')
- `autoai_model_metric` - the AutoAI model evaluation result (as float, e.g. '0.7')
- `reference_model` - the reference model (as CPD Path, e.g. 'cpd:///spaces/b5ad9f7e-eec3-49ad-a8f6-86354ff29134/assets/b6046bcd-7c20-4fc8-bb70-9ed0944d185b')
- `reference_model_metric` - the reference model evaluation result (as float, e.g. '0.67')


In [17]:
AUTOAI_MODEL = os.getenv('autoai_model')
AUTOAI_MODEL_METRIC = float(os.getenv('autoai_model_metric'))
REFERENCE_MODEL = os.getenv('reference_model')
REFERENCE_MODEL_METRIC = float(os.getenv('reference_model_metric'))

REFERENCE_MODEL_ID = CpdPath.from_string(REFERENCE_MODEL).resource_id()
AUTOAI_MODEL_ID = CpdPath.from_string(AUTOAI_MODEL).resource_id()

print('AutoAI model:', AUTOAI_MODEL_ID)
print('AutoAI model metric:', AUTOAI_MODEL_METRIC)
print('Reference model:', REFERENCE_MODEL_ID)
print('Reference model metric:', REFERENCE_MODEL_METRIC)

AutoAI model: b6046bcd-7c20-4fc8-bb70-9ed0944d185b
AutoAI model metric: 0.7
Reference model: aa374e8b-dc3e-426a-a3c2-96dc1e006c80
Reference model metric: 0.67


## Connect to Watsonx.ai Runtime

Select the deployment space where the reference model, pointed in `reference_model` environment variable, is stored.

In [22]:
pipelines_client = WatsonPipelines.from_apikey(api_key, url=cpdaas_url)

# Extract space ID from reference model CPD Path passed from the Watson Pipeline as an environment variable
space_scope = CpdPath.from_string(REFERENCE_MODEL)
space_id = space_scope.scope_id()


In [24]:
# Initiate Watsonx.ai Client
credentials = Credentials(
    api_key=api_key,
    url='https://' + location + '.ml.cloud.ibm.com'
)

client = APIClient(credentials)

client.set.default_space(space_id)

'SUCCESS'

<a id="selection"></a>
## 2. Model selection

The best model is selected based on an evaluation metric value.

In [25]:
if AUTOAI_MODEL_METRIC > REFERENCE_MODEL_METRIC:
    print('Selected AutoAI model')
    selected_model_id = AUTOAI_MODEL_ID
    selected_model = AUTOAI_MODEL
else:
    print('Selected reference model')
    selected_model_id = REFERENCE_MODEL_ID
    selected_model = REFERENCE_MODEL

Selected AutoAI model


Mark selected model with `selected_model` tag.

In [26]:
not_seleceted_metadata = {
    client.repository.ModelMetaNames.TAGS: []
}
for model in client.repository.get_model_details()['resources']:
    model_id = client.repository.get_model_id(model)
    client.repository.update_model(model_id, not_seleceted_metadata)

selected_metadata = {
    client.repository.ModelMetaNames.TAGS: ["selected_model"]
}
client.repository.update_model(selected_model_id, selected_metadata)

print('Models tagged.')

Models tagged.


<a id="results"></a>
## 3. Save notebook results

This notebook is run from the Orchestration Pipeline that expects following resuklts:

- `selected_model` - the selected model CPD Path


In [27]:
results = {
  "selected_model": selected_model
}
print(results)

{'selected_model': 'cpd:///spaces/b5ad9f7e-eec3-49ad-a8f6-86354ff29134/assets/b6046bcd-7c20-4fc8-bb70-9ed0944d185b'}


Finally, store the results, which are consumed by a flow downstream nodes, using the Watson Pipelines client.

In [ ]:
res = pipelines_client.store_results(results)

print('DONE')

### Authors

**Rafał Bigaj**, System Architect with long successful record of building and leading teams. Broad and practical knowledge in the area of cloud computing, machine learning and distributed systems development.

**Krzysztof Niedziela**, Automation Developer in Cloud Pack for Data.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.